# <center> MyFitnessPal data pipeline

In [14]:
# importing all the dependencies
import myfitnesspal
import datetime
from datetime import date, timedelta, datetime
import pandas as pd
import os
import numpy as np


os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')
import basic_methods as bm

In [16]:
mfp_exec = pd.read_csv('/Users/jayrajparmar/Documents/side_project/health_data_tracking/myfitnesspal/File-Export-2017-09-22-to-2022-06-01/Exercise-Summary-2017-09-22-to-2022-06-01.csv')
mfp_nutrition = pd.read_csv('/Users/jayrajparmar/Documents/side_project/health_data_tracking/myfitnesspal/File-Export-2017-09-22-to-2022-06-01/Nutrition-Summary-2017-09-22-to-2022-06-01.csv')


In [17]:
# Function to clean column names, convert date to datetime and delete Remarks field
def data_prep(df, date_col):
    """ 
    This function does the following:
            * Column name cleaning: Removes characters from column names
            * Datetime conversion: Converts date column to datetime
            * Null value Imputation: Imputes null values with "-1"
    Takes 2 inputs:
            * df = DataFrame
            * date_col: column name for dates
    """
#     # Column name cleaning
    d = {' ':'_',
         '(':'',
         ')':''}
    df.columns = [i.replace(' ','_').replace('(','').replace(')','') for i in list(df)]
    print(df.columns)
    df = bm.datetime_conversion(df, [date_col])
    df = df.fillna(-1)
    return df


# Creating a function to convert features to numeric and extracting just date from datetime
def date_numeric_groupby(df, date_col):
    """
    First part is for converting datatypes to appropriate type for aggregation without error
    Second part is for removing multiple entries for same dates
    Takes 2 inputs:
            * df = DataFrame
            * date_col: column name for dates
       """
    df = df.convert_dtypes()
    try:
        lis=[]
        for i in df.columns:
            
            if df[i].dtypes == 'datetime64[ns]':
                lis.append(df[i].dt.date)
                print(str(i)+str('(')+str(df[i].dtypes)+str(')')+str(':')+'Done')
            elif df[i].dtypes == str:
                lis.append(df[i].astype(str))
                print(str(i)+str('(')+str(df[i].dtypes)+str(')')+str(':')+'Done')
            else: 
                lis.append(df[i].apply(pd.to_numeric, errors='coerce'))
                print(str(i)+str('(')+str(df[i].dtypes)+str(')')+str(': ')+'Done')
    except:
        print("There is an unknown datatype in the data!!")
    df_temp = pd.concat(lis, axis=1)
    print('************************************************************************************')
    print('************************************************************************************')
    print('*****************************Datatypes after conversion*****************************')
    print('************************************************************************************')
    print('************************************************************************************')

    print(df_temp.dtypes)
    
    # Creating list of numerical columns
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    col_lis = list(df_temp.select_dtypes(include=numerics).columns)

    # Creating a list of np.max functions for creating dictionary
    max_func = [np.sum for i in range(len(col_lis))]

    # Creating dictionary from both lists
    d = dict(zip(col_lis, max_func))

    df_max_cols = df_temp.groupby([date_col]).agg(d).reset_index()

    print('Shape of dataframe before removing duplicates: ' + str(df_temp.shape))

    print('Shape of dataframe after removing duplicates: ' + str(df_max_cols.shape))
    return df_max_cols

def null_imputation_rolling(df,date_col):
    """
    This function converts date column to datetime and creates rolling 7 day 
    averages for all numeric columns
    Takes 2 inputs:
        * df = DataFrame
        * date_col: column name for dates
    """
    df = bm.datetime_conversion(df, [date_col])
    df = df.sort_values(date_col)
    # Now we would fill the null values using .interpolate with linear method; it would be somewhat like imputing
    # mean values from previous and next value in columns
    df = df.set_index(date_col)
#     df.interpolate(method='linear', limit_direction='forward', axis=0, inplace=True)
    for i in df.columns:
        if i != date_col:
            df[i+str('_rolling_7')] = df[i].rolling(window=7).mean()
    return df


def date_range_generator(start_dt, end_dt):
    """
    Function for generating date range between start and end date;
    It has 2 arguments to be added:
        * start_dt = datetime value for the start date
        * end_dt = datetime value for the end date
    """
    lis = []
    while start_dt < end_dt:
        start_dt += timedelta(days=1)
        lis.append(start_dt)
    df = pd.DataFrame(lis, columns=['date_range'])
    return df

In [18]:
mfp_exec_cleaned1 = data_prep(mfp_exec, 'Date')
mfp_exec_cleaned2 = date_numeric_groupby(mfp_exec_cleaned1, 'Date')
del mfp_exec_cleaned2['Exercise']
del mfp_exec_cleaned2['Type']

mfp_exec_cleaned3 = null_imputation_rolling(mfp_exec_cleaned2,'Date')

total_dates = date_range_generator(min(mfp_exec_cleaned3.index), datetime.today().date())

final_df_exec = total_dates.merge(mfp_exec_cleaned3, how='left', 
                             left_on='date_range', right_on=mfp_exec_cleaned3.index)



Index(['Date', 'Exercise', 'Type', 'Exercise_Calories', 'Exercise_Minutes',
       'Sets', 'Reps_Per_Set', 'Kilograms', 'Steps', 'Note'],
      dtype='object')
Date(datetime64[ns]):Done
Exercise(string): Done
Type(string): Done
Exercise_Calories(Int64): Done
Exercise_Minutes(Int64): Done
Sets(Int64): Done
Reps_Per_Set(Int64): Done
Kilograms(Int64): Done
Steps(Int64): Done
Note(Int64): Done
************************************************************************************
************************************************************************************
*****************************Datatypes after conversion*****************************
************************************************************************************
************************************************************************************
Date                  object
Exercise             float64
Type                 float64
Exercise_Calories      int64
Exercise_Minutes       int64
Sets                   int64
Reps_Per

In [20]:
mfp_nutrition_cleaned1 = data_prep(mfp_nutrition, 'Date')
mfp_nutrition_cleaned2 = date_numeric_groupby(mfp_nutrition_cleaned1, 'Date')

del mfp_nutrition_cleaned2['Note']

mfp_nutrition_cleaned3 = null_imputation_rolling(mfp_nutrition_cleaned2,'Date')


total_dates = date_range_generator(min(mfp_nutrition_cleaned3.index), datetime.today().date())

final_df_nutrition = total_dates.merge(mfp_nutrition_cleaned3, how='left', 
                             left_on='date_range', right_on=mfp_nutrition_cleaned3.index)


Index(['Date', 'Meal', 'Calories', 'Fat_g', 'Saturated_Fat',
       'Polyunsaturated_Fat', 'Monounsaturated_Fat', 'Trans_Fat',
       'Cholesterol', 'Sodium_mg', 'Potassium', 'Carbohydrates_g', 'Fiber',
       'Sugar', 'Protein_g', 'Vitamin_A', 'Vitamin_C', 'Calcium', 'Iron',
       'Note'],
      dtype='object')
Date(datetime64[ns]):Done
Meal(string): Done
Calories(Float64): Done
Fat_g(Float64): Done
Saturated_Fat(Float64): Done
Polyunsaturated_Fat(Float64): Done
Monounsaturated_Fat(Float64): Done
Trans_Fat(Float64): Done
Cholesterol(Float64): Done
Sodium_mg(Float64): Done
Potassium(Float64): Done
Carbohydrates_g(Float64): Done
Fiber(Float64): Done
Sugar(Float64): Done
Protein_g(Float64): Done
Vitamin_A(Float64): Done
Vitamin_C(Float64): Done
Calcium(Float64): Done
Iron(Float64): Done
Note(Int64): Done
************************************************************************************
************************************************************************************
***************